In [25]:
import re
import wave
import pyaudio
import _thread
import time

# https://www.youtube.com/watch?v=KSSsVhoR7FQ

In [ ]:
class TextToSpeech:
    
    CHUNK = 1024
    
    def __init__(self, words_pron_dict:str = 'cmudict-0.7b.txt'):
        self._l = {}
        self._load_words(words_pron_dict)
        
    def _load_words(self, words_pron_dict:str):
        '''
            Extract words to use as the key, then put each sound
            into a list to the value.
        '''
        with open(words_pron_dict, 'r') as file:
            for line in file:
                if not line.startswith(';;;'):
                    key, val = line.split(' ', 2)
                    self._l[key] = re.findall(r'[A-Z]+', val)
    
    def get_pronunciation(self, str_input):
        '''
            Loads each sound of the input into a a sort of queue,
            to play the corresponding audio file while also incrementing
            a delay to make the output speech a little more fluid. 
        '''
        list_pron = []
        for word in re.findall(r"[\w']+", str_input.upper()):
            if word in self._l:
                list_pron += self._l[word]
        print(list_pron)
        delay=0
        for pron in list_pron:
            _thread.start_new_thread( TextToSpeech._play_audio, (pron,delay,))
            delay += 0.145
    
    def _play_audio(sound, delay):
        '''
        Handles the delay and plays a sound.
        This method is based off the sample function on the PI audio
        project webpage
        '''
        try:
            time.sleep(delay)
            wf = wave.open('sounds/'+sound+'.wav', 'rb')
            p = pyaudio.PyAudio()
            stream = p.open(format=p.get_format_from_with(wf.getsampwidth()),
                            channels=wf.getnchannels(),
                            rate=wf.getframerate(),
                            output=True)
            data = wf.readframes(TextToSpeech.CHUNK)
            
            while data:
                stream.write(data)
                data = wf.readframes(TextToSpeech.CHUNK)
            
            stream.stop_stream()
            stream.close()
            
            p.terminate()
            return
        except:
            pass:
            
if __name__ == '__main__':
    tts = TextToSpeech()
    while True:
        tts.get_pronunciation(input('Enter the fucking sentense: '))
                    